In [1]:
import os
import numpy as np
import glob
from matplotlib import pyplot as plt

In [2]:
import rasterio
import earthpy.plot as ep
from patchify import patchify
import tifffile as tifffile
import random

In [3]:
# check working directory
os.getcwd()

'/data/private/Thesis/May'

### **Patching Single Datasets**

In [4]:
# read val image

with rasterio.open("Data/02_Tile/label_bin_gaveau/label_bin_gaveau11.tif") as src:
    lbl = src.read()

print(lbl.shape)

(1, 5903, 6055)


In [5]:
lbl = np.transpose(lbl, (1,2,0))

In [6]:
PATCH_SIZE = 256
N_BANDS_LBL = lbl.shape[-1]

print(N_BANDS_LBL)

1


In [7]:
patches_msk = patchify(lbl, (PATCH_SIZE, PATCH_SIZE, N_BANDS_LBL), step=PATCH_SIZE)

if not os.path.exists('Valmsk'):
    os.makedirs('Valmsk')

for i in range(patches_msk.shape[0]):
    for j in range(patches_msk.shape[1]):
        
        single_patch_msk = patches_msk[i,j,:,:]
        tifffile.imwrite('Valmsk/' + 'valmsk_' + str(i) + '_' + str(j) + ".tif", single_patch_msk)

In [8]:
msk_patch = sorted(glob.glob('Valmsk/*.tif'))

print(len(msk_patch))

529


In [9]:
y = np.zeros((len(msk_patch), PATCH_SIZE, PATCH_SIZE, N_BANDS_LBL))

for i, (msk) in enumerate(zip(msk_patch)):
    msk = tifffile.imread(msk)
    np_msk = np.array(msk)
    y[i] = np_msk

In [10]:
print(y.shape)

(529, 256, 256, 1)


In [11]:
if not os.path.exists('Data/03_Patch/val'):
    os.makedirs('Data/03_Patch/val')

np.save('Data/03_Patch/val/ygaveau.npy', y)